In [1]:
pip install faker

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.6 MB 11.1 MB/s            
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import datetime
import random
from faker import Faker

In [19]:
fake = Faker()
Faker.seed(42)
random.seed(42)

In [20]:
df = pd.read_csv("https://raw.githubusercontent.com/stepthom/869_course/main/data/GermanCredit.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 62 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Duration                                1000 non-null   int64 
 1   Amount                                  1000 non-null   int64 
 2   InstallmentRatePercentage               1000 non-null   int64 
 3   ResidenceDuration                       1000 non-null   int64 
 4   Age                                     1000 non-null   int64 
 5   NumberExistingCredits                   1000 non-null   int64 
 6   NumberPeopleMaintenance                 1000 non-null   int64 
 7   Telephone                               1000 non-null   int64 
 8   ForeignWorker                           1000 non-null   int64 
 9   Class                                   1000 non-null   object
 10  CheckingAccountStatus.lt.0              1000 non-null   int64 
 11  Check

In [21]:
today = datetime.date(2021, 6, 1)
print(today)

2021-06-01


In [22]:
dob = today - datetime.timedelta(days=40*365)
print(dob)

1981-06-11


In [23]:
df.head()

,Duration,Amount,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance,Telephone,ForeignWorker,Class,...,OtherInstallmentPlans.Bank,OtherInstallmentPlans.Stores,OtherInstallmentPlans.None,Housing.Rent,Housing.Own,Housing.ForFree,Job.UnemployedUnskilled,Job.UnskilledResident,Job.SkilledEmployee,Job.Management.SelfEmp.HighlyQualified
0,6,1169,4,4,67,2,1,0,1,Good,...,0,0,1,0,1,0,0,0,1,0
1,48,5951,2,2,22,1,1,1,1,Bad,...,0,0,1,0,1,0,0,0,1,0
2,12,2096,2,3,49,1,2,1,1,Good,...,0,0,1,0,1,0,0,1,0,0
3,42,7882,2,4,45,1,2,1,1,Good,...,0,0,1,0,0,1,0,0,1,0
4,24,4870,3,4,53,2,2,1,1,Bad,...,0,0,1,0,0,1,0,0,1,0


In [24]:
#Generate 20 random cities:
n = 20
cities = [fake.city() for _ in range(20)]
print(cities)
cities_probs = [1.0/(i+1) for i in range(20)]
cities_probs_norm =  [float(i)/sum(cities_probs) for i in cities_probs]
print(cities_probs_norm)


['North Judithbury', 'East Jill', 'New Roberttown', 'East Jessetown', 'Lake Debra', 'Robinsonshire', 'Lisatown', 'Lake Roberto', 'Ericmouth', 'North Noahstad', 'Cassandraton', 'Herrerafurt', 'New Kellystad', 'Lake Chad', 'Port Keith', 'Port Jesseville', 'Ramirezstad', 'West Michael', 'Jacquelineland', 'New Jessica']
[0.2779522965244017, 0.13897614826220084, 0.09265076550813389, 0.06948807413110042, 0.055590459304880345, 0.046325382754066946, 0.039707470932057384, 0.03474403706555021, 0.0308835885027113, 0.027795229652440172, 0.025268390593127427, 0.023162691377033473, 0.02138094588649244, 0.019853735466028692, 0.01853015310162678, 0.017372018532775105, 0.01635013508967069, 0.01544179425135565, 0.014629068238126404, 0.013897614826220086]


In [25]:

for i in df.index:
    
    # Age
    days = df.loc[i, 'Age']*365 + (random.randrange(365*2) - 365)
    dob = today - datetime.timedelta(days=int(days))
    df.at[i, 'DateOfBirth'] = dob
    
    if df.loc[i, 'Personal.Male.Divorced.Seperated'] == 1 or df.loc[i, 'Personal.Male.Single'] == 1 or df.loc[i, 'Personal.Male.Married.Widowed'] == 1:
        df.at[i, 'Sex'] = "M"
        df.at[i, 'FirstName'] = fake.first_name_male()
    else:
        df.at[i, 'Sex'] = "F"
        df.at[i, 'FirstName'] = fake.first_name_female()
        
    df.at[i, 'LastName'] = fake.last_name()
        
    if df.loc[i, 'Personal.Male.Married.Widowed'] == 1 or df.loc[i, 'Personal.Female.NotSingle'] == 1:
        df.at[i, 'Married'] = int(1)
    else:
        df.at[i, 'Married'] = int(0)
    

    # Nonsense feature
    df.loc[i, 'NumberPets'] = random.randrange(3)
    
    if df.loc[i, 'EmploymentDuration.lt.1'] == 1:
        df.at[i, 'EmploymentDuration'] = 0
    elif df.loc[i, 'EmploymentDuration.1.to.4'] == 1:
        df.at[i, 'EmploymentDuration'] = 1 + random.randrange(4)
    elif df.loc[i, 'EmploymentDuration.4.to.7'] == 1:
        df.at[i, 'EmploymentDuration'] = 4 + random.randrange(4)
    elif df.loc[i, 'EmploymentDuration.gt.7'] == 1:
        df.at[i, 'EmploymentDuration'] = 7 + random.randrange(10)
        
    if df.loc[i, 'SavingsAccountBonds.Unknown'] == 1:
        df.at[i, 'SavingsAccountBonds'] = random.randrange(600)
    elif df.loc[i, 'SavingsAccountBonds.lt.100'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 0
    elif df.loc[i, 'SavingsAccountBonds.100.to.500'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 100 + random.randrange(401)
    elif df.loc[i, 'SavingsAccountBonds.500.to.1000'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 500 + random.randrange(501)
    elif df.loc[i, 'SavingsAccountBonds.gt.1000'] == 1:
        df.at[i, 'SavingsAccountBonds'] = 1000 + random.randrange(1000)
        
        
    inflation = 1.8
    
    df.at[i, 'Amount'] = round(df.loc[i, 'Amount'] * inflation)
    
   
    df.at[i, 'Street'] = fake.street_address()
    #df.at[i, 'City'] = fake.city()
    
    df.at[i, "LicensePlate"] = fake.license_plate()
    
    df.at[i, 'UserID'] = fake.ssn()
    
df['City'] = random.choices(cities, cities_probs_norm, k=df.shape[0])

In [26]:
df = df.drop(['Age'], axis=1)
df = df.rename(columns={'Telephone': 'OwnCar', 'Class': "Credit"})
df = df[df.columns.drop(list(df.filter(regex='Personal.')))]
df = df[df.columns.drop(list(df.filter(regex='EmploymentDuration.')))]
df = df[df.columns.drop(list(df.filter(regex='SavingsAccountBonds.')))]

In [27]:
df['BadCredit'] = df['Credit'].apply(lambda x: 0 if x == 'Good' else 1)
df = df.drop('Credit', axis=1)

In [28]:
front_cols = ['UserID', 'FirstName', 'LastName', 'DateOfBirth', 'Sex', 'Street', 'City', 'LicensePlate', 'Married', "NumberPets"]

df = df[ front_cols + [ col for col in df.columns if col not in front_cols] ]

last_cols = ['BadCredit']
df = df[ [ col for col in df.columns if col not in last_cols] + last_cols ]

In [29]:
df.head()

,UserID,FirstName,LastName,DateOfBirth,Sex,Street,City,LicensePlate,Married,NumberPets,...,Housing.Rent,Housing.Own,Housing.ForFree,Job.UnemployedUnskilled,Job.UnskilledResident,Job.SkilledEmployee,Job.Management.SelfEmp.HighlyQualified,EmploymentDuration,SavingsAccountBonds,BadCredit
0,218-84-8180,Christopher,Gray,1953-09-02,M,503 Linda Locks,North Judithbury,395C,0.0,0.0,...,0,1,0,0,0,1,0,7.0,281.0,0
1,643-21-6917,Jennifer,Rocha,1999-09-30,F,42388 Burgess Meadow Suite 532,East Jill,012 PCY,1.0,0.0,...,0,1,0,0,0,1,0,2.0,0.0,1
2,520-14-4890,Kyle,Cruz,1973-03-01,M,480 Erin Plain Suite 514,West Michael,7-F0482,0.0,2.0,...,0,1,0,0,1,0,0,4.0,0.0,0
3,081-11-7963,Ryan,Romero,1975-10-17,M,52880 Burns Creek,North Judithbury,30Z J39,0.0,1.0,...,0,0,1,0,0,1,0,4.0,0.0,0
4,463-16-4062,Robert,Spence,1969-05-15,M,78248 Brandt Plains,Ramirezstad,3-46578,0.0,0.0,...,0,0,1,0,0,1,0,2.0,0.0,1


In [30]:
df['BadCredit'].value_counts()

0    700
1    300
Name: BadCredit, dtype: int64

In [31]:
df.to_csv('generated_german.csv', index=False)

# Now make a not-OHE version

In [32]:
df2 = df.copy(deep=True)

In [33]:
df2.head().T

,0,1,2,3,4
UserID,218-84-8180,643-21-6917,520-14-4890,081-11-7963,463-16-4062
FirstName,Christopher,Jennifer,Kyle,Ryan,Robert
LastName,Gray,Rocha,Cruz,Romero,Spence
DateOfBirth,1953-09-02,1999-09-30,1973-03-01,1975-10-17,1969-05-15
Sex,M,F,M,M,M
Street,503 Linda Locks,42388 Burgess Meadow Suite 532,480 Erin Plain Suite 514,52880 Burns Creek,78248 Brandt Plains
City,North Judithbury,East Jill,West Michael,North Judithbury,Ramirezstad
LicensePlate,395C,012 PCY,7-F0482,30Z J39,3-46578
Married,0,1,0,0,0
NumberPets,0,0,2,1,0


In [34]:
df2['Married'] = df2['Married'].map({0:'no', 1:'yes'})
df2['OwnCar'] = df2['OwnCar'].map({0:'no', 1:'yes'})
df2['ForeignWorker'] = df2['ForeignWorker'].map({0:'no', 1:'yes'})

In [17]:
for t in ['Purpose', 'Property', 'Job', 'Housing', 'OtherDebtorsGuarantors', 'CreditHistory', 'CheckingAccountStatus', 'OtherInstallmentPlans']:
  cols = [c for c in df2.columns.tolist() if t in c]
  df2[t] = df2[cols].idxmax(axis=1)
  df2 = df2.drop(cols, axis=1)

df2.head()

TypeError: reduction operation 'argmax' not allowed for this dtype